In [1]:
import zipfile
import wget
import pandas as pd
import glob
import numpy as np
import re
from sklearn import linear_model
import matplotlib.pyplot as plt

# Get the building ID

In [2]:
df = pd.read_csv('../data/UCL/id_UCL.csv')

# Download data

In [3]:
# ids=[]
# for i in df['id']:
#     for year in [2017,2018]:
#         try : 
#             wget.download(f'https://platform.carbonculture.net/apps/studydata/data-download?year={year}&id={i}&subject=places')
#             ids.append((id,year))
#             print("ID ", i, " Year ", year, " exists")
#         except:
#             print("ID ", i, " Year ", year, " doesn't exist")
#             break

In [4]:
# file_list = glob.glob("./*.zip")

# for file in file_list:
#     with zipfile.ZipFile(file, 'r') as zip_ref:
#         zip_ref.extractall('./extracted/')
#         print(file, "extracted!")

In [5]:
file_list = glob.glob("../data/UCL/UCL_extracted/*.csv")

dfs = []

for file in file_list:
    df = pd.read_csv(file, skiprows=3, index_col=0)
    df = df.stack().reset_index()
    df.columns = ['date','time','meter_reading']
    df['timestamp'] = pd.to_datetime(df['date'] + ' ' + df['time'])
    df['site_id'] = re.findall('\.[0-9]{4}_',file)[0][1:5]
    df['type'] = re.findall('_[a-z]{4}\.csv',file)[0][1:5]
    
    print("File has been read and added", file)
    
    dfs.append(df)

File has been read and added ../data/UCL/UCL_extracted/places.1161_2017_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1132_2018_heat.csv
File has been read and added ../data/UCL/UCL_extracted/places.1161_2016_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1117_2018_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1139_2018_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1184_2018_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1158_2017_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1158_2016_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1193_2018_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1140_2017_heat.csv
File has been read and added ../data/UCL/UCL_extracted/places.1113_2018_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1140_2016_heat.csv
File has been read and added

File has been read and added ../data/UCL/UCL_extracted/places.1188_2017_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1188_2016_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1154_2018_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1202_2018_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1119_2018_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1215_2018_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1156_2016_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1156_2017_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1173_2016_heat.csv
File has been read and added ../data/UCL/UCL_extracted/places.1173_2017_heat.csv
File has been read and added ../data/UCL/UCL_extracted/places.1211_2018_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1138_2018_heat.csv
File has been read and added

File has been read and added ../data/UCL/UCL_extracted/places.1173_2017_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1173_2016_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1112_2018_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1137_2018_heat.csv
File has been read and added ../data/UCL/UCL_extracted/places.1159_2017_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1159_2016_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1192_2018_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1185_2018_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1138_2018_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1211_2018_heat.csv
File has been read and added ../data/UCL/UCL_extracted/places.1160_2017_elec.csv
File has been read and added ../data/UCL/UCL_extracted/places.1133_2018_heat.csv
File has been read and added

In [6]:
dfs = pd.concat(dfs,axis=0)
dfs['id'] = dfs['site_id'] + '_' + dfs['type']
ucl_data = pd.pivot_table(dfs,index=["timestamp"],columns=['id'], values=['meter_reading'])
ucl_data.columns = [col[1] for col in ucl_data.columns]
ucl_data = ucl_data.resample('1H').sum()

# Aim is to match UCL building to site 1 buildings

In [11]:
train = pd.read_csv('../data/ashrae/train.csv')

d_types = {'building_id': np.int16,
          'meter': np.int8}

for feature in d_types:
    train[feature] = train[feature].astype(d_types[feature])
    
train["timestamp"] = pd.to_datetime(train["timestamp"])

In [12]:
building = pd.read_csv('../data/ashrae/building_metadata.csv')
d_types = {'building_id': np.int16,
          'site_id': np.int8,
          'primary_use': 'category',
          'square_feet': np.int32,
          'year_built': np.float16,
          'floor_count': np.float16}

for feature in d_types:
    building[feature] = building[feature].astype(d_types[feature])
    
building['site_id'] = building['site_id'].replace({8:0,11:7})

In [13]:
train = pd.merge(train,building, how='left')
train = train[train['site_id']==1]
train['id'] = train['building_id'].astype(str) + '_' + train['meter'].astype(str)

In [14]:
train_pt = pd.pivot_table(train,index=["timestamp"],columns=['id'], values=['meter_reading'])
train_pt.columns = [col[1] for col in train_pt.columns]

In [15]:
corr_mat = {}
for site_id in train_pt.columns:
    subset = train_pt[[site_id]].join(ucl_data)
    corr_mat[site_id] = subset.corr()[site_id].iloc[1:]
    
corr_mat = pd.DataFrame(corr_mat)

In [16]:
mapping = pd.DataFrame([])
mapping['UCL_id'] = corr_mat.apply(lambda x: np.argmax(x), axis=0)
mapping['score'] = corr_mat.apply(lambda x: max(x), axis=0)

/anaconda3/envs/kaggle/lib/python3.7/site-packages/numpy/core/fromnumeric.py:61: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return bound(*args, **kwds)


In [23]:
ucl_data = ucl_data[mapping['UCL_id']]
ucl_data.columns = mapping.index
ucl_data = ucl_data.stack().reset_index()
ucl_data.columns = ['timestamp','level_1','meter_reading']
ucl_data['building_id']=ucl_data['level_1'].str.split('_').apply(lambda x: x[0])
ucl_data['meter']=ucl_data['level_1'].str.split('_').apply(lambda x: x[1])
ucl_data = ucl_data[['timestamp','building_id','meter','meter_reading']]

In [25]:
ucl_data.to_csv('../data/UCL/ucl_data_prepared.csv',index=False)